In [1]:
from core.utils import Tibanna
from core import ff_utils

#format for input json in hic-partA
def make_input_file_json(obj_id, arg_name, tibanna):
    '''
    {
      "bucket_name": "%s",
      "object_key": "%s",
      "uuid" : "%s",
      "workflow_argument_name": "%s"
    }
    '''
    ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
    metadata = ff_utils.get_metadata(obj_id, connection=ff)
    data = {}
    
    # just make sure the file is on s3, otherwise bail
    print("looking for upload key %s, on bucket %s" % 
          (metadata['upload_key'],
           tibanna.s3.outfile_bucket))
    if tibanna.s3.does_key_exist(metadata['upload_key']):
        data = {'bucket_name' : tibanna.s3.outfile_bucket,
                'object_key' : metadata['upload_key'].split('/')[1],
                'uuid' : metadata['uuid'],
                'workflow_argument_name': arg_name
                }
    return data
    

def make_hic1_json(input_files, env, output_bucket, accession):
    input_json = {'input_files': input_files,
                  'output_bucket': output_bucket,
                  'workflow_uuid': "0fbe4db8-0b5f-448e-8b58-3f8c84baabf5",
                  "app_name": "bwa-mem",
                  "parameters": {
                      "nThreads": 32
                      },
                  "config": {
                      "ebs_type": "io1",
                      "json_bucket": "4dn-aws-pipeline-run-json",
                      "ebs_iops": 500,
                      "shutdown_min": 30,
                      "s3_access_arn": "arn:aws:iam::643366669028:instance-profile/S3_access",
                      "ami_id": "ami-cfb14bb5",
                      "copy_to_s3": True,
                      "script_url": "https://raw.githubusercontent.com/4dn-dcic/tibanna/master/awsf/",
                      "launch_instance": True,
                      "password": "hahaha",
                      "log_bucket": "tibanna-output",
                      "key_name": "4dn-encode"
                    },
                  "_tibanna": {"env": env, "run_type": "bwa-mem",
                               "run_id": accession}
                  }
    return input_json
    

In [4]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

all_sets = [
            'dcic:Selvaraj_gm12878_hic',
            'dekker-lab:ExperimentSet_U54_U54-ESC4DN-FA-DpnII-2017524',
            'dekker-lab:ExperimentSet_U54_HFFc6-FA-DpnII'
            ]
   
my_rep_set = all_sets[2]
print my_rep_set
exclude_miseq = True

wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1/1"

wf_partA = "awesem-hi-c-processing-parta-juicer/25"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")

# status for completion
# there are two flavors of complete signals, before it was output_file_transfer_finished, not it is complete.
# old completed wf runs have former one.
status_done = ['complete', 'output_file_transfer_finished']

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
rep_resp = ff_utils.get_metadata(my_rep_set, connection=ff)['experiments_in_set']
################
##ADD TO WORKFLOW
# wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
# run_hours = int((datetime.now()-wfr_time).total_seconds()/3600)
################

def summarize_file(file_resp):
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')
    # get related file
    paired_file = ''
    for relation in relations:
        if relation['relationship_type'] == 'paired with':
            paired_file = relation['file']['accession']
    
    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    last_part_A = ''
    last_part_A_status = 'did_not_run'
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)  
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_partA):
                last_part_A=wfr_resp['uuid']
                last_part_A_status = wfr_resp.get('run_status')  
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # if exclude miseq is on, do this only if sequencer is not miseq
    if not exclude_miseq or sequencer != "Illumina MiSeq":
        # check if md5 step is completed properly
        
        if status != "uploaded" or md5_status not in status_done:
            # if not, shall we run it?
            if run_md_qc in ['md5', 'all']:
                print 'md5 running for', file_resp['accession']
                code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                run(code_md5)
                print ''
                time.sleep(10)
            # user does not want it to be run, so just report
            else:
                print 'md5 run missing for', file_resp['accession']
        # check fastqc if md5 is fine
        else:
            if not qc or fastqc_status not in status_done:
                # if not, shall we run it?
                if run_md_qc in ['qc', 'all']:
                    print 'fastqc running for', file_resp['accession']
                    code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                    run(code_qc)
                    print ''    
                    time.sleep(10)
                # user does not want it to be run, so just report
                else:
                    print 'fastqc run missing for', file_resp['accession'], fastqc_status
                    print 
   
    # return a small report
    return {'file': file_id,
            'alias': first_alias,
            'sequencer': sequencer,
            'pair_no': pair_no,
            'paired_file': paired_file,
            'file_status': status,
            'qc': qc,
            'md5_status': md5_status,
            'fastqc_status': fastqc_status,
            'last_part_A': last_part_A,
            'last_part_A_status': last_part_A_status
           }


report = []
enzymes = []
for exp in rep_resp:
    # print 'Experiment', exp
    exp_resp = ff_utils.get_metadata(exp, connection=ff)
    exp_files = exp_resp['files']
    enzyme = exp_resp['digestion_enzyme']
    enzymes.append(enzyme)
    for fastq_file in exp_files:
        file_resp = ff_utils.get_metadata(fastq_file, connection=ff, frame='embedded')  
        #Some checks before running
        #check if status is deleted
        if file_resp['status'] == 'deleted':
            continue
        #if no uploaded file in the file item report and skip
        if not file_resp.get('filename'):
            print file_resp['accession'], "does not have a file"
            continue
        # check if file is in s3
        head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket)
        if not head_info:
            print file_resp['accession'], "does not have a file in S3"
            continue
        
        # skip pair no 2
        if file_resp.get('paired_end')=='2':
            continue 
            
        # if the file size is more than 42 GB do not run
        file_size = head_info['ContentLength']
        if file_size > 45097156608:
            print file_resp['aliases'], 'larger than 42GB, skipping, please use another script'
            continue
        filesize = round(head_info['ContentLength']/1073741824.0,1)

        file_info = summarize_file(file_resp)
 
        # check for miseq
        if exclude_miseq:
            if file_info['sequencer'] == 'Illumina MiSeq':
                continue
        paired_file = file_info['paired_file']
        pair_file_resp = ff_utils.get_metadata(paired_file, connection=ff, frame='embedded')
        pair_file_info = summarize_file(pair_file_resp)
        print fastq_file,paired_file, filesize
        print

        # check consistency of paired file info
        # status differences gives error but there are multiple statuses that indicate complete
        # TODO fix it
        pairs_inconsistent = ""
        check_items = [ i for i in file_info.keys() if i not in ['file', 'paired_file', 'pair_no', 'alias']]
        for check_item in check_items:
            try:
                assert file_info[check_item] == pair_file_info[check_item]
            except AssertionError:
                print check_item, "not the same between pair", fastq_file, 'and', paired_file
                pairs_inconsistent += check_item + ', '
        wf_check = ''
        # check if md5 and qc are okay
        for info in [file_info, pair_file_info]:
            if (info['md5_status'] in status_done and info['file_status'] == 'uploaded' and
                info['fastqc_status'] in status_done and info['qc'] == True):
                wf_check += '+'
        rep = {"consistency": pairs_inconsistent,  
               "file1": file_info['alias'],
               "file2": pair_file_info['alias'],
               "const_check": pairs_inconsistent,
               "wf_check": wf_check, 
               "partA_wf": file_info['last_part_A'], 
               "partA_status": file_info['last_part_A_status']
               }
        # status differences gives error but there are multiple statuses that indicate complete
        # TODO fix it
        report.append(rep)
        if rep.get('const_check'):
            print rep['const_check']

    
# TODO need to add failed ones
# 1 completed pairs
pairs_completed = [i for i in report if i['partA_status']=='complete']
# 2 running pairs
pairs_running = [i for i in report if i['partA_status'] not in ['complete','did_not_run']]
# 3 no run pairs
pairs_did_not_run = [i for i in report if i['partA_status']=='did_not_run']
# 3a no run pairs with fine qc md5
pairs_ready_to_run = [(i['file1'], i['file2']) for i in pairs_did_not_run if i['wf_check'] == '++']
# 3b no run pairs with problematic qc md5
pairs_qcmd_problem = [(i['file1'], i['file2']) for i in pairs_did_not_run if i['wf_check'] != '++']

# 2 running pairs to run again
rerun_running_pairs = [(i['file1'], i['file2']) for i in report if i['partA_status'] not in ['complete','did_not_run']]

rerun_started_pairs = [(i['file1'], i['file2']) for i in report if i['partA_status']=='started']


print "{}/{} pairs completed partA".format(len(report), len(pairs_completed))
print "{}/{} pairs still running partA".format(len(report), len(pairs_running))

print pairs_running
print ",".join(i['partA_status'] for i in pairs_running)
print ''
print '1) ready to run (pairs_ready_to_run)'
for a,b in pairs_ready_to_run:
    print a,b
print ""

print '2) problematics ones (pairs_qcmd_problem)'
for a,b in pairs_qcmd_problem:
    print a,b
print ""

print '3) running ones (rerun_running_pairs)'
for a,b in rerun_running_pairs:
    print a,b
print ""

print '4) running ones (rerun_started_pairs)'
for a,b in rerun_started_pairs:
    print a,b
print ""

# # Choose the right NZ reference file
# re_ref_file = ''
# choice = {'HindIII': '4DNFI823MBKE', 'MboI': '4DNFI823L812'}
# # Check if all experiments use the same enzyme
# if len(list(set(enzymes))) != 1:
#     print "ERROR Mixed Enzyme Content in Experiment Set"
# else:
#     nz_name = enzymes[0].split('/')[2]
#     re_ref_file = choice[nz_name]
# print 'using {} ({}) as the enzyme'.format(nz_name, re_ref_file)
print "DONE"


dekker-lab:ExperimentSet_U54_HFFc6-FA-DpnII
Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)
md5 run missing for 4DNFIWDY2U4S
md5 run missing for 4DNFIS8I1Y5L
/files-fastq/4DNFIWDY2U4S/ 4DNFIS8I1Y5L 10.2

md5 run missing for 4DNFI65A88XH
md5 run missing for 4DNFIY67X18K
/files-fastq/4DNFI65A88XH/ 4DNFIY67X18K 10.0

md5 run missing for 4DNFIONTKGCC
md5 run missing for 4DNFISXNGXNA
/files-fastq/4DNFIONTKGCC/ 4DNFISXNGXNA 10.1

md5 run missing for 4DNFIAAJYJ1Y
md5 run missing for 4DNFICOIG8A1
/files-fastq/4DNFIAAJYJ1Y/ 4DNFICOIG8A1 10.3

md5 run missing for 4DNFINCPY93G
md5 run missing for 4DNFI6BFAKQI
/files-fastq/4DNFINCPY93G/ 4DNFI6BFAKQI 10.3

md5 run missing for 4DNFIF5LBBWQ
md5 run missing for 4DNFI9S272R7
/files-fastq/4DNFIF5LBBWQ/ 4DNFI9S272R7 10.1

md5 run missing for 4DNFID9WXC6L
md5 run missing for 4DNFIWIGZ5EW
/files-fastq/4DNFID9WXC6L/ 4DNFIWIGZ5EW 9.9

md5 run missing for 4DNFIPC1UAH2
md5 run missing for 4DNFIG2BLIPZ
/files-fastq/4DNFIPC1UAH2/ 4DNFIG2BLIPZ 10

In [5]:
from core.utils import Tibanna
from core.utils import run_workflow
import time



paired_files = pairs_qcmd_problem

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
outfiles = tibanna.s3.outfile_bucket
tibanna.s3.outfile_bucket = 'elasticbeanstalk-fourfront-webprod-files'

# todo need a function to determin this given fastq1
index = make_input_file_json('4DNFIZQZ39L9', 'bwa_index', tibanna)

for pair in paired_files:
    fastq1 = make_input_file_json(pair[0], 'fastq1', tibanna)
    fastq2 = make_input_file_json(pair[1], 'fastq2', tibanna)
    
    input_files = [fastq1, fastq2, index]
    if all(input_files):
        name = fastq1['object_key'].split('.')[0] + "-" + fastq2['object_key'].split('.')[0]
        input_json = make_hic1_json(input_files, env, outfiles, name)
        #print input_json
        res = run_workflow(input_json)
    else:
        print("some files not found on s3.  Investigate this list %s" % input_files)
    #time.sleep(5)
    a = raw_input("Press Enter to continue...")

print('Done')


looking for upload key 1f53df95-4cf3-41cc-971d-81bb16c486dd/4DNFIZQZ39L9.bwaIndex.tgz, on bucket elasticbeanstalk-fourfront-webprod-files
looking for upload key 01795b9f-b843-4ac6-8391-7ef2b0497782/4DNFIWDY2U4S.fastq.gz, on bucket elasticbeanstalk-fourfront-webprod-files
looking for upload key d9f827b1-d3c6-4139-8287-e8e0fedccc5e/4DNFIS8I1Y5L.fastq.gz, on bucket elasticbeanstalk-fourfront-webprod-files
about to start run bwa-mem_4DNFIWDY2U4S-4DNFIS8I1Y5L
response from aws was: 
 {u'startDate': datetime.datetime(2017, 9, 15, 14, 24, 53, 115000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '2ae0c80c-9a43-11e7-ae49-bfbb466a1cac', 'HTTPHeaders': {'x-amzn-requestid': '2ae0c80c-9a43-11e7-ae49-bfbb466a1cac', 'content-length': '160', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_awsem_workflow_with_ponies:bwa-mem_4DNFIWDY2U4S-4DNFIS8I1Y5L'}
url to view status:
https://con

Press Enter to continue...
looking for upload key 61fcc2de-ff3c-499b-ade1-f7ad343f2f89/4DNFIFXD526I.fastq.gz, on bucket elasticbeanstalk-fourfront-webprod-files
looking for upload key 80fb2618-14c7-4e2e-9479-a9105b158d24/4DNFI77ORE8G.fastq.gz, on bucket elasticbeanstalk-fourfront-webprod-files
about to start run bwa-mem_4DNFIFXD526I-4DNFI77ORE8G
response from aws was: 
 {u'startDate': datetime.datetime(2017, 9, 15, 14, 25, 56, 985000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '50f2434f-9a43-11e7-8a45-7f4874d3af45', 'HTTPHeaders': {'x-amzn-requestid': '50f2434f-9a43-11e7-8a45-7f4874d3af45', 'content-length': '160', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_awsem_workflow_with_ponies:bwa-mem_4DNFIFXD526I-4DNFI77ORE8G'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:exec

Press Enter to continue...
looking for upload key 0642ed53-3c66-4d3c-85b5-b899606e4bcc/4DNFIMXEJLHF.fastq.gz, on bucket elasticbeanstalk-fourfront-webprod-files
looking for upload key 830517db-e20c-4b6b-8b87-8480dfdc40c1/4DNFIFDD2YM2.fastq.gz, on bucket elasticbeanstalk-fourfront-webprod-files
about to start run bwa-mem_4DNFIMXEJLHF-4DNFIFDD2YM2
response from aws was: 
 {u'startDate': datetime.datetime(2017, 9, 15, 14, 26, 41, 261000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '6b555465-9a43-11e7-a528-3d96a552c320', 'HTTPHeaders': {'x-amzn-requestid': '6b555465-9a43-11e7-a528-3d96a552c320', 'content-length': '160', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_awsem_workflow_with_ponies:bwa-mem_4DNFIMXEJLHF-4DNFIFDD2YM2'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:exec